In [49]:
%matplotlib inline
import pandas as pd
import subprocess
import itertools
import numpy as np

# Configuracion
command = "./tp {input} {output} {method}"

tests = [
    {'input': 'experimento1-1',
     'output': 'experimento1-1-outputs/{method}-{h}.test',
     'method': [0, 1],
     'skip': 1,
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento1-2',
     'output': 'experimento1-2-outputs/{method}-{h}.test',
     'method': [0, 1],
     'skip': 1,
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento1-3',
     'output': 'experimento1-3-outputs/{method}-{h}.test',
     'skip': 1,
     'method': [0, 1],
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento2-1',
     'output': 'experimento2-1-outputs/{name}-{method}-{h}.test',
     'method': [0, 1],
     'runs': 5,
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento3-1',
     'inputs': ['experimento3-1-0', 'experimento3-1-1', 'experimento3-1-2', 'experimento3-1-3', 'experimento3-1-4', 'experimento3-1-5', 'experimento3-1-6', 'experimento3-1-7', 'experimento3-1-8', 'experimento3-1-9', 'experimento3-1-10'],
     'output': 'experimento3-1-outputs/{name}-{method}-{h}.test',
     'skip': 1,
     'method': [2, 3],
     'h': [1.0]}]

# Generacion de datos
for file in tests:
    name = file['input']
    
    dataset = []
    
    try:
        if file['skip']:
            print 'Salteando experimento', name
            continue
    except:
        pass
    
    print 'Comenzando experimento', name
    
    try:
        names = file['inputs']
    except:
        names = [name]
        
    for nameI in names:
        print 'Corriendo subexperimento', nameI
        
        with open(nameI + '.input', 'r') as handle:
            file['contents'] = handle.read()
            print nameI + '.input'

            for (run_method, run_h) in itertools.product(file['method'], file['h']):
                output = {}
                output['name'] = nameI
                output['method'] = run_method
                output['h'] = run_h

                with open('/tmp/tmp.test', 'w+') as writer:
                    writer.write(file['contents'].format(h=run_h))

                outFile = file['output'].format(name=nameI, h=run_h, method=run_method)

                try:
                    runs = file['runs']
                except:
                    runs = 1
                
                for r in xrange(runs):
                    fmt = command.format(input='/tmp/tmp.test', output=outFile, method=run_method)
                    runtime = subprocess.check_output(fmt, shell=True)

                    for line in runtime.split('\n'):
                        if line:
                            line = line.split(':')
                            line[0] = line[0].strip().lower()
                            line[1] = float(line[1].strip())
                            output[line[0]] = line[1]

                    dataset.append(output)

    dataset = pd.DataFrame(dataset)
    dataset.to_csv(name + '.csv', index=False)
    print 'CSV guardado'
        
    print 'Experimento', name, 'finalizado'

Salteando experimento experimento1-1
Salteando experimento experimento1-2
Salteando experimento experimento1-3
Comenzando experimento experimento2-1
Corriendo subexperimento experimento2-1
experimento2-1.input
CSV guardado
Experimento experimento2-1 finalizado
Salteando experimento experimento3-1


In [ ]:
# Experimento 1-1
# Muchas sanguijuelas pequeñas cerca del punto crítico, la idea es que a medida que aumentemos la granularidad
# van a empezar a aparecer en la discretización, haciendo que la temperatura del punto crítico vaya tornándose
# cada vez más alta.
import numpy.random as npr

n=50
width=100
height=100

minX = width/2 - 5
maxX = width/2 + 5
minY = height/2 - 5
maxY = height/2 + 5

posX = npr.uniform(minX, maxX, n)
posY = npr.uniform(minY, maxY, n)
radius = map(abs, npr.normal(0.5, max([maxX - minX, maxY - minY]), n))
temperatures = npr.exponential(100, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

In [41]:
# Experimento 1-2
# Sanguijuelas alejadas, de un radio comparativamente grande.
# Lo que debería suceder en este caso es que no se modificaría la temperatura del punto crítico a medida
# que incrementemos la granularidad.
import numpy.random as npr

n=50
width=100
height=100

posX = npr.uniform(0, width, n)
posY = npr.uniform(0, height, n)
radius = map(abs, npr.normal(2, max(width, height), n))
temperatures = npr.exponential(100, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

100 100 {h} 50
79.3561607453 83.680565098 21.8926178367 27.0666829694
71.8577712936 19.6928171382 166.86017519 241.23178885
5.14222103842 45.1434035333 65.8709658048 24.8271940482
79.1129433607 87.4846188438 180.833275962 21.5333100325
12.9224577894 11.1954609767 15.8968786747 43.9834876836
50.4283457225 47.0806998726 74.1709565317 212.805137914
29.4081708934 16.0481296234 102.0955005 56.8422814762
80.7106288871 28.2487032858 225.498169093 4.91879826156
66.6047089711 40.475125353 33.237016507 158.224779622
17.9872079396 77.2791720129 72.3645038813 0.0830761516577
32.8984423176 36.0607721138 35.1616722861 32.8207048294
88.3971303773 43.6341663027 176.631201389 12.6989095169
16.5668065511 44.288988627 125.359867141 84.7921917889
9.42574244528 71.281707878 63.0140503838 184.339099355
3.24781920029 69.1635064774 18.0498903958 79.7265391333
40.9774684362 6.40342476556 42.3820269842 88.7556750335
40.7320174192 31.2865967877 50.5406841841 64.4325755341
18.2819721646 2.3442241482 8.752337606 1

In [46]:
# Experimento 1-3
# Tomar sanguijuelas pequeñas, alejadas del centro, con temperaturas muy altas.
# Si las temperaturas son lo suficientemente altas se deberían propagar hacia el centro a medida que aumente h
import numpy.random as npr

n=50
width=100
height=100

posX = npr.uniform(0, width, n)
posY = npr.uniform(0, height, n)
radius = map(abs, npr.normal(0.1, max(width, height)/10, n))
temperatures = npr.exponential(300, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

100 100 {h} 50
87.8131384076 17.3743081829 5.73615868377 576.337008441
90.2781668696 5.24508212808 1.81964603385 76.2431044719
65.3618021328 11.375172959 1.45094692367 87.2814044611
64.3101571641 13.1852400663 0.534245476475 5.98167480845
83.6750880921 87.3654766957 7.20118335367 79.3006920091
51.5117669215 6.53788650521 6.52795636262 266.341013647
8.06570855215 33.3447044735 2.63463625991 47.8107136898
92.3462701233 87.7829020355 13.6318176083 357.512841177
29.1118349348 67.3581030912 6.97746141911 735.661424007
49.3937267561 48.3279703449 3.21969446022 80.5866940278
4.11242677742 24.3586437528 6.21617968217 436.053302475
40.8673346921 56.8708775479 7.04834758818 118.272967712
73.1079254191 59.8292435649 8.58119314757 242.992243142
96.8282054853 68.0208529005 8.6017977606 40.1282769055
43.9683113483 41.5808301386 18.712011732 214.152725807
79.8499774816 18.6411006852 16.3544032708 814.969505304
2.2942675584 99.2866689282 5.76899836394 308.465849381
93.8201093057 48.0319370886 4.409481

In [48]:
# Experimento 2-1
# Este experimento es solo para ver que pasa con las sanguis
import numpy.random as npr

n=50
width=100
height=100

posX = npr.uniform(0, 10, n)
posY = npr.uniform(0, 10, n)
radius = npr.uniform(0, 10, n)
temperatures = npr.exponential(300, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

100 100 {h} 50
7.96062242745 3.54857786448 4.14168055012 557.127075854
1.16566228323 0.186102467612 2.61466037666 358.361125268
4.91139503158 5.48481445315 4.32788596694 346.996765583
7.52329514018 0.816429277384 3.66052615763 946.890652378
0.558422341451 1.7685836137 3.1888511243 938.456694511
8.04755943902 7.01844378145 3.17536333641 143.538041595
6.07072988977 4.22902249525 6.63023938832 82.741541444
1.67205428972 6.04933717617 8.49387547503 244.847945942
6.10811936833 0.254961853132 0.571922270307 146.373424831
5.37672916457 6.94821613251 4.77398305045 55.629966592
2.90891051241 5.21747166323 4.59491906464 211.679515907
6.306869581 9.84357032452 4.98474073407 303.109097423
2.72098831785 8.55822470112 2.6589328347 5.38661897824
3.56617161521 2.54773366236 9.86965616174 569.073811236
4.96783575664 1.1035867631 6.16266645352 151.728667601
6.57660899209 0.566944558264 8.29997405458 9.35982329837
2.06712310387 0.98033697762 0.126947119907 778.216007498
2.36688814064 7.83731075143 6.2119

In [12]:
# Experimento 3-1
# Tomamos una grilla de sanguijuelas situadas en los puntos de la discretización (no variamos el h)
# y vamos aumentando la cantidad de sanguijuelas unitarias (es decir, las que tienen radio < h).

# IMPORTANTE: Si se cambia el h, cambiarlo en el primer casillero también
import numpy as np
import numpy.random as npr
import copy
import itertools

width=10
height=10
h = 0.2
percentages = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

posX = list(np.arange(0, width, h))
posY = list(np.arange(0, height, h))

# Aca comienza el codigo

# Cantidad de leeches totales
n = len(posX)*len(posY)
radiusLower = list(map(abs, npr.uniform(-h, 0, n)))
radiusHigher = list(npr.uniform(h, width, n))
temperatures = list(npr.exponential(300, n))
positions = list(itertools.product(posX, posY))

for x in percentages:
    print width, height, "{h}", n
    leftPositions = copy.deepcopy(positions)
    leftRL = copy.deepcopy(radiusLower)
    leftRH = copy.deepcopy(radiusHigher)
    leftT = copy.deepcopy(temperatures)

    unitarias = round(x*float(n))
    noUnitarias = float(n) - unitarias
    
    i = 0
    while i < unitarias:
        # Generamos la cantidad de sanguijuelas unitarias que hagan falta
        (x, y) = leftPositions.pop()
        r = leftRL.pop()
        t = leftT.pop()
        
        print x, y, r, t
        i += 1.0
    
    i = 0
    while i < noUnitarias:
        # El resto son sanguijuelas no unitarias
        (x, y) = leftPositions.pop()
        r = leftRH.pop()
        t = leftT.pop()
        
        print x, y, r, t
        i += 1.0

10 10 {h} 2500
9.8 9.8 3.84930759645 345.377580195
9.8 9.6 4.11099010465 13.0814447953
9.8 9.4 5.13017892808 224.615120605
9.8 9.2 6.24105405852 450.236913291
9.8 9.0 8.86544674935 86.3456355852
9.8 8.8 2.62492908849 3.23613426189
9.8 8.6 9.34162676984 224.206136499
9.8 8.4 5.55451448588 1023.90867103
9.8 8.2 6.27154034704 320.120155284
9.8 8.0 8.3140179328 93.1928606815
9.8 7.8 6.20041054709 318.874398133
9.8 7.6 6.83826528928 197.584758561
9.8 7.4 0.331794595659 397.226379938
9.8 7.2 4.77535088153 154.237210468
9.8 7.0 3.60714093897 507.090973789
9.8 6.8 8.62469727462 51.60972866
9.8 6.6 2.55409367434 119.749121409
9.8 6.4 4.6544520947 159.450777234
9.8 6.2 4.01221467983 147.742046602
9.8 6.0 5.31891515314 103.155302261
9.8 5.8 8.01234284863 152.214203126
9.8 5.6 5.78535208457 344.632286958
9.8 5.4 7.49089820982 215.20976999
9.8 5.2 0.497507068862 621.090573035
9.8 5.0 6.18834032758 945.385679235
9.8 4.8 6.85493000187 0.451768245975
9.8 4.6 6.09203677862 33.0310885806
9.8 4.4 6.54606

In [36]:
# Selección de los h para la experimentación, queremos ver qué h sirven con el width y height que buscamos.
# Si hay más de 50 000 de dims, va a ser complicado que el experimento corra bien, asi que el h tiene que
# estar por debajo de eso para que la cosa ande.
width=100.0
height=100.0

lst = [x for x in np.arange(0.01, 100, 0.01) if round(width/x) == width/x and round(height/x) == height/x]
[(x, (width/x)*(height/x)) for x in lst]

[(0.01, 100000000.0),
 (0.02, 25000000.0),
 (0.040000000000000001, 6250000.0),
 (0.050000000000000003, 4000000.0),
 (0.080000000000000002, 1562500.0),
 (0.16, 390625.0),
 (0.20000000000000001, 250000.0),
 (0.25, 160000.0),
 (0.40000000000000002, 62500.0),
 (0.5, 40000.0),
 (0.80000000000000004, 15625.0),
 (1.0, 10000.0),
 (1.25, 6400.0),
 (2.0, 2500.0),
 (2.5, 1600.0),
 (4.0, 625.0),
 (5.0, 400.0),
 (6.25, 256.0),
 (10.0, 100.0),
 (12.5, 64.0),
 (50.0, 4.0)]

In [38]:
map(lambda (x, y): x, [(0.5, 40000.0), (0.80000000000000004, 15625.0), (1.0, 10000.0), (1.25, 6400.0), (2.0, 2500.0), (2.5, 1600.0), (4.0, 625.0), (5.0, 400.0), (6.25, 256.0), (10.0, 100.0), (12.5, 64.0), (50.0, 4.0)])

[0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5, 50.0]

In [27]:
10000.0*10000.0

100000000.0